#### 

#### conda install -c pytorch pytorch

In [1]:
from elements.yolo import LINE_DETECTION
import numpy as np
import cv2
import os
from glob import glob
from time import time
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line as phl
from copy import deepcopy
import math
from sklearn.mixture import GaussianMixture as GMM
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

In [2]:
def rad(degree):
    return (degree * np.pi/180)

In [3]:
def line_detector_func(image, edges, theta_range=None):
    line_image = 0 * image
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # canny edge detector filter
    lines = phl(edges,
                 threshold=threshold,
                 line_length=max(image_gray.shape)//2,
                 line_gap=max(image_gray.shape)//2,
                 theta=theta_range,
                 seed=2022)


    idx = 0
    while len(lines)>1 and idx<len(lines)-1:
        (x00, y00), (x01, y01) = lines[idx]
        (x10, y10), (x11, y11) = lines[idx+1]
        m0 = (y01-y00) / (x01-x00)
        m1 = (y11-y10) / (x11-x10)
        if np.abs(m1-m0) <= tetha_thr:
            lines.pop(idx+1)
        else:
            idx+=1
    polar= []
    for ((x0, y0), (x1, y1)) in lines: 
        thetar = np.arctan((y1+y0)/(x1+x0))
        rho = np.sqrt(((x0+x1)/2)**2+((y0+y1)/2)**2)
        polar.append([rho,thetar])
        cv2.line(line_image, (x0, y0), (x1, y1), (255, 255, 255), thickness=thickness)
    polar = np.array(polar)  
    return line_image, lines, polar

### Hyperparameters

In [4]:
theta = np.concatenate([np.arange(start=rad(-60),stop=rad(-5),step=rad(1)),np.arange(start=rad(5),stop=rad(60),step=rad(1))])
tetha_thr = 0.5
thickness = 2
region_thr = 1000
# canny hyoerparameters
sigma = 5
low_threshold = 1
high_threshold = 25
# probabilistic_hough_line hyperparameters
threshold = 10

### Violations, if camera has rear view of the cars, it most be [0,1]. for front view of cars it will be [-2,-1]

In [5]:
# for line_sebghat viokation
line_num = [0,1]

### Image Path

In [6]:
image = '/Users/Sean/Desktop/FYP/Code/Traffic-Law-Enforcement-main/test.jpg'
# load image
frame = cv2.imread(image)

### Detection Process

In [7]:
# detector objects
line_detector = LINE_DETECTION('./weights/line_model.pt')

Fusing layers... 


In [2]:
# detection process, detecting lines
lines = line_detector.detect(frame)
bb_mask = np.zeros((frame.shape))
# extract edges from whole image
image_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
# canny edge detector filter
edges = canny(image_gray,sigma=sigma,low_threshold=low_threshold,high_threshold=high_threshold)
edges_3ch = np.zeros((frame.shape))
edges_m = np.zeros((frame.shape))
centre = []
for line in lines:
    [(xmin,ymin),(xmax,ymax)] = line['bbox']
    [(xc,yc)]= line['centre']
    centre.append([xc,yc])
    for row in range(int(ymin),int(ymax)):
        for col in range(int(xmin),int(xmax)):
            if edges[row,col] == True:
                edges_3ch[row,col,:] = 255
centre = np.array(centre)
# cv2.imwrite('03_edges.jpg', cv2.resize(edges_3ch,(edges.shape[1],edges.shape[0])))
print(centre)



NameError: name 'line_detector' is not defined

In [9]:
# from skimage import morphology as morph
# ar = edges_3ch[:,:,1] >0
# new_edges = morph.remove_small_objects(ar, 100, connectivity = 2)
# newedges_3ch = np.zeros((frame.shape))
# for row in range(new_edges.shape[0]):
#     for col in range(new_edges.shape[1]):
#         if new_edges[row,col] == True:
#             newedges_3ch[row,col,:] = 255
#         else:
#             newedges_3ch[row,col,:] = 0
            
# cv2.imwrite('03_edges.jpg', cv2.resize(newedges_3ch,(edges.shape[1],edges.shape[0])))

In [10]:
# line_image, lines, polar = line_detector_func(frame,new_edges,theta_range=None)

# cv2.imwrite('05_lines.jpg', cv2.resize(line_image,(edges.shape[1],edges.shape[0])))

In [11]:
# gray = cv2.cvtColor(line_image, cv2.COLOR_BGR2GRAY)

# cv2.imwrite('lines.jpg', cv2.resize(gray,(edges.shape[1],edges.shape[0])))

In [12]:
# lines = polar.reshape(polar.shape[0],2)
# print(centre)
# print(lines.shape)

In [ ]:
#preprocessing features to be in (0-1) range
# scaler = MinMaxScaler()
# scaler.fit(centre)
# centre = scaler.fit_transform(centre)
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
gm = GMM(n_components=2, random_state=0).fit(X)
# gmm = GMM(n_components=5)
# gmm.fit(centre)
# labels = gmm.predict(centre)

# centre = scaler.inverse_transform(centre) #getting back our original values

# num_clusters = np.max(labels) + 1
# print(num_clusters, "clusters detected")

# for i, line in enumerate(centre):
#     if labels[i] == 0:
#         color = 'g'
#     elif labels[i] == 1:
#         color = 'b'
#     elif labels[i] == 2:
#         color = 'r'
#     elif labels[i] == 3:
#         color = 'y'
#     elif labels[i] == 4:
#         color = 'm'
#     elif labels[i] == 5:
#         color = 'c'
#     elif labels[i] == -1:
#         color = 'k'
    
    
#     plt.scatter(line[0],line[1],c=color)
    
# plt.show()

# grouped = defaultdict(list)
# #grouping lines by clusters
# for i, label in enumerate(labels):
#     grouped[label].append([centre[i,0],centre[i,1]])

# means = []
# #getting mean values by cluster
# for i in range(num_clusters):
#     mean = np.mean(np.array(grouped[i]), axis=0)
#     means.append(mean)
# means = np.array(means)

# #black dots are classified as noise

In [14]:
# image = '/Users/Sean/Desktop/FYP/Code/Traffic-Law-Enforcement-main/test.jpg'
# # load image
# frame = cv2.imread(image)
# img = deepcopy(frame)
# print(means)
# for rho, theta in means:
#     a = math.cos(theta)
#     b = math.sin(theta)
#     x0 = a * rho
#     y0 = b * rho
#     pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
#     pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
#     img = cv2.line(img, pt1, pt2, (0,0,255), 2, cv2.LINE_AA)

# cv2.imwrite('cluster.jpg', cv2.resize(img,(img.shape[1],img.shape[0])))